In [1]:
import pandas as pd
import numpy as np

## load data

In [3]:
y = pd.read_csv("cls_labelWithfea.csv")
print(len(y.loc[y["label"] == 0]))
print(len(y.loc[y["label"] == 1]))
y = y["label"]

32827
2418


In [12]:
fea_concat = pd.read_csv("sym_mesh_concat.csv",index_col=0)
print(fea_concat.shape)
fea_concat.head()

(35245, 692)


,0,1,2,3,4,5,6,7,8,9,...,38.1.1,39.1.1,40.1.1,41.1.1,42.1.1,43.1.1,44.1.1,45.1.1,46.1.1,47.1.1
0,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve

from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

from sklearn.utils import shuffle

from imblearn.under_sampling import RandomUnderSampler # Random UnderSampler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve,auc

## 负采样+划分测试集

In [18]:
import random as python_random
seed = 1
np.random.seed(seed) # This is necessary for starting Numpy generated random numbers in a well-defined initial state.
python_random.seed(seed)# This is necessary for starting core Python generated random numbers in a well-defined state.

rus = RandomUnderSampler(sampling_strategy=1,random_state=seed) 
rus_x, rus_y = rus.fit_resample(fea_concat ,y) #rus_y(Series)
print('负采样后数据集大小:',len(rus_y))

fea_train,fea_test,y_train,y_test=train_test_split(
                rus_x,
                rus_y,
                test_size=0.2, shuffle=True)

print(fea_train.shape,fea_test.shape)

负采样后数据集大小: 4836
(3868, 692) (968, 692)


## RF

### GridSearchCV

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

clf=RandomForestClassifier(class_weight={1:1,0:1},
                           random_state=seed)

kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)

trees=[2000]
maxdepths=[10]
min_samples_leafs=[5]
min_samples_splits=[10]
#max_featuress=list(range(20,41,2))#default 25
param_grid=dict(n_estimators= trees,
                max_depth=maxdepths,
                min_samples_leaf=min_samples_leafs,
                min_samples_split=min_samples_splits)

scorings=["accuracy","roc_auc"]

grid = GridSearchCV(clf, param_grid, cv=kfold, 
                    scoring=scorings,return_train_score = True,refit="roc_auc",
                   verbose=2)
grid_result=grid.fit(fea_train,y_train)

# GridSearchCV results
params = grid_result.cv_results_["params"]
mean_train_accuracy=grid_result.cv_results_["mean_train_accuracy"]
mean_train_roc_auc=grid_result.cv_results_["mean_train_roc_auc"]

mean_test_accuracy=grid_result.cv_results_["mean_test_accuracy"]
mean_test_roc_auc=grid_result.cv_results_["mean_test_roc_auc"]

print("train_acc:",mean_train_accuracy)
print("train_auc:",mean_train_roc_auc)
print("test_acc:",mean_test_accuracy)
print("test_auc:",mean_test_roc_auc)

for mean_train_acc,mean_test_acc,mean_train_auc,mean_test_auc,param in zip(
                                        mean_train_accuracy,mean_test_accuracy,
                                        mean_train_roc_auc,mean_test_roc_auc,params):
    print("train_acc:{}---test_acc:{}  with:   {}\n" .format(mean_train_acc,mean_test_acc,param),"\n",
         "train_auc:{}---test_auc:{}  with:   {}\n" .format(mean_train_auc,mean_test_auc,param))
    
print("\n grid.best_score_:",grid.best_score_ ,"\n best_params_:",grid.best_params_)
#print(grid_result.cv_results_.keys())

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=10, n_estimators=2000; total time=  16.2s
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=10, n_estimators=2000; total time=  16.4s
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=10, n_estimators=2000; total time=  16.2s
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=10, n_estimators=2000; total time=  16.3s
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=10, n_estimators=2000; total time=  16.2s
train_acc: [0.83628523]
train_auc: [0.92195991]
test_acc: [0.71561452]
test_auc: [0.78130718]
train_acc:0.8362852276489072---test_acc:0.7156145224318153  with:   {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 2000}
 
 train_auc:0.9219599108893094---test_auc:0.7813071780315515  with:   {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 2000}


 grid.best_scor

min_samples_split=100,
min_samples_leaf=20,

grid.best_score_: 0.7672469198360907 
best_params_: {'max_depth': 20, 'n_estimators': 200}
    
    #############
min_samples_split=50,
min_samples_leaf=10,
    
grid.best_score_: 0.7828178384472568 
best_params_: {'max_depth': 50, 'n_estimators': 100}
    
    
    #################
grid.best_score_: 0.7763511180871401 
best_params_: {'max_depth': 50, 'min_samples_leaf': 5, 'min_samples_split': 100, 'n_estimators': 300}

      ############
grid.best_score_: 0.7865976307593436 
best_params_: {'max_depth': 50, 'min_samples_leaf': 5, 'min_samples_split': 50, 'n_estimators': 700}

       #####
grid.best_score_: 0.7952418093456814 
best_params_: {'max_depth': 50, 'min_samples_leaf': 5, 'min_samples_split': 20, 'n_estimators': 2000}

    ######不太过拟合
grid.best_score_: 0.7813071780315515 
best_params_: {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 2000}

### Test

In [86]:
#y_test-binary-int;y_pred-proba-float
def test_metric(y_test,y_pred):
    y_test=np.array(y_test)
    y_pred=np.array(y_pred)
    
    y_pred_bina = np.around(y_pred,0).astype(int)
    class_acc=accuracy_score(y_test,y_pred_bina)
    
    y_test_int = y_test.astype(int)
    fpr,tpr,threshold=roc_curve(y_test_int,y_pred)#y_test-int(ndarray)
    print(len(fpr))
    roc_auc=auc(fpr,tpr)
    
    print('accuracy {}'.format(str(class_acc)), 'AUC:{}'.format(roc_auc))

    return np.array(class_acc),np.array(roc_auc)


In [88]:
clf=RandomForestClassifier(n_estimators=2000,
                           max_depth=10,
                           min_samples_leaf=5,
                           min_samples_split=10,
                           class_weight={1:1,0:1})
clf.fit(fea_train,y_train)

predictions=[i[1] for i in clf.predict_proba(fea_test)]

accuracy,AUC = test_metric(y_test,predictions)

print('predictions lenth：{}\n'.format(len(predictions)),
    'accuracy {} \n'.format(accuracy),
     'AUC {} \n'.format(AUC))


388
accuracy 0.6911157024793388 AUC:0.7648338032982995
predictions lenth：968
 accuracy 0.6911157024793388 
 AUC 0.7648338032982995 



In [92]:
AUC_data = np.array([list(y_test),list(predictions)])
print(AUC_data.shape)
AUC_data_df=pd.DataFrame(AUC_data.T,columns=["label","predict"])
AUC_data_df.head()

(2, 968)


,label,predict
0,0.0,0.381249
1,0.0,0.519821
2,0.0,0.390581
3,1.0,0.654111
4,0.0,0.500389


In [94]:
# AUC_data_df.to_csv("auc_data_rf.csv")